### Hash function


In [1]:
import time
import numpy as np
import random 

In [2]:
def hash_fun(string):
    m=2**32
    hash_= 7
    for i in range(len(string)):
        x = encoding(string[i])
        hash_=((hash_*23))+x
    hash_=hash_%m #to avoid overflow
    binary = bin(hash_)[2:]
    str_binary = str(binary)
    return str_binary

In [3]:
def encoding(string):
    s=0
    for char in string:
        s+=ord(char) #convert each str into ASCII char
    return s

### Hyperloglog

### Add item to the buckets: 
The add operation consists of computing the hash of the input data v with a hash function h, getting the first b bits (where b is ${\textstyle \log _{2}(m)}$), and adding 1 to them to obtain the address of the register to modify. With the remaining bits compute ${\textstyle \rho (w)}$ which returns the position of the leftmost 1. The new value of the register will be the maximum between the current value of the register and ${\textstyle \rho (w)}$.

In [4]:
def leftmost_zeros(string):
    maximum=0
    temp=0
    for i in range(len(string)):
        if string[i]=="0":
            temp+=1
        else:
            maximum=temp
            break
    return maximum

In [5]:
def hll(file):
    length_file = 139000000     
    f = open(file, "r") #open the file
    n=13
    m = 2**n
    #initialize the buckets
    buckets = [0 for x in range (m)]
    #for each rows in the file
    for row in range(0,length_file):
        #read one line at time
        line = f.readline(row).strip()
        #compute the hash function on it and save the value in digit
        digit = hash_fun(line) 
        if len(digit)<32: #filling with zeros in order to obtain digit of 32 bit
            digit = '0'*(32-len(digit))+digit
             
        #add items to the buckets
        j = int(digit[:n],2) 
        w=digit[n:] 
        buckets[j]=max(buckets[j],leftmost_zeros(w))
           
    #computing cardinality 
    alfa = 0.7213/(1+1.079/(2**n))
    Z=0
    for i in range(len(buckets)):
        Z+=2**(-buckets[i]-1)
    Z=Z**(-1)
    E = alfa*(m)**2*Z 
    return (E)

### Cardinality of the hyperloglog:

The cardinality of the hyperloglog is given by:
${\displaystyle E=\alpha _{m}m^{2}Z}$ 


Where Z is: 
${\displaystyle Z={\Bigg (}\sum _{j=1}^{m}{2^{-M[j]}}{\Bigg )}^{-1}}$ 


and $\alpha _{m}$ (for $m>=128$) is: $0.7213/(1+1.079/(2^{m}))$

In [6]:
file = "hash.txt" 
card = hll(file)
print('Cardinality: ', card)

Cardinality:  121891579.94193806


### Error of the hyperloglog:
In order to see the error of our hyperloglog we can compare the value of distinct elements in the file hash.txt and the obtained cardinality.

In [10]:
real_len = 125000001 # obtained with set()
#error = (real_len-card/real_len)*100
error = real_len/(abs(real_len-card)/card)*100
print('Error: ',error)

Error:  490166786610.74164
